In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage import io
from torchvision.transforms import v2
import torchvision
import random
from pathlib import Path

In [2]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

True
cuda


In [3]:
aug_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomAffine(degrees=(0,5), translate=(0.1, 0.3), scale=(0.5, 0.75)),
    transforms.ToTensor()
])

In [4]:
root_dir = '/HDD-1T/KNN'

In [5]:
desired_images_per_class = 300

In [6]:
class TransformedImageFolderDataset(Dataset):
    def __init__(self, root, transform=None):
        self.image_folder = torchvision.datasets.ImageFolder(root=root)
        self.transform = transform

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        image, label = self.image_folder[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [7]:
dataset = TransformedImageFolderDataset(root=root_dir, transform=aug_transforms)

In [8]:
# Iterate over classes and augment images
for class_name in dataset.image_folder.classes:
    # Create the directory for the class if it doesn't exist
    class_dir = os.path.join(root_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)
    
     # Determine how many images are needed for this class
    existing_images = [file for file in os.listdir(class_dir) if file.endswith('.jpg')]
    images_needed = desired_images_per_class - len(existing_images)
    
    # Augment existing images to reach the desired number
    for i in range(images_needed):
        # Randomly select an existing image
        image_name = random.choice(existing_images)
        image_path = os.path.join(class_dir, image_name)
        
        # Load and augment the image
        image = Image.open(image_path)
        transformed_image = aug_transforms(image)
        
        # Convert the tensor back to PIL image before saving
        transformed_image_pil = transforms.functional.to_pil_image(transformed_image)
        
        # Save the augmented image
        transformed_image_pil.save(os.path.join(class_dir, f'{len(existing_images) + i}.jpg'))

        
print("Augmentation completed.")

Augmentation completed.


In [ ]:
orig_img = Image.open(Path('/HDD-1T/KNN/F_RIGHT/F.30.jpg'))
affine_imgs = [aug_transforms(orig_img) for _ in range(4)]
img_index = 0
for img in affine_imgs:
    transformed_image_pil = transforms.functional.to_pil_image(img)
    transformed_image_pil.save(os.path.join('/home/antonia/Desktop/TEST KNN', f'{img_index}.jpg'))
    img_index+=1